<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/UI/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📦 Project Overview

This Google Colab notebook contains the core code for the **Phoenix Team Project**, a comprehensive system designed for IoT data management, knowledge sharing, and user engagement. The project is built using Python and the Gradio library for creating interactive user interfaces.

## ⚙️ Setup and Initialization

The initial cell handles essential setup:
- **Package Installation:** Installs necessary libraries like `gradio`, `pandas`, and `importnb`.
- **Repository Cloning:** Clones the project's GitHub repository (`Cloud-Course-Group-Phoenix/Project-Pheonix.git`) to access project modules.
- **Branch Checkout:** Switches to the `Dev` branch of the repository.
- **Python Path:** Adds the project's `Logic` directory to the Python path, allowing easy import of custom modules (`Indexmqtt`, `CloudDB`, `Admin`, `SearchService`, `UserManager`, `TaskLogic`, `SensorVisualLogic`, `ChatbotLogic`).
- **Module Import:** Imports all the required project modules.

This ensures all necessary components and dependencies are in place before the application starts.

## 🖼️ User Interface Components

The notebook defines several functions to create different parts of the user interface using Gradio:

- **`create_admin_dashboard()`:** Builds the UI for the administrative panel, including features for managing search index status, re-indexing content, and viewing top search terms. It also includes sections for managing daily tasks for the system.

- **`create_admin_tasks_dashboard()`:** Creates a dedicated admin interface for managing all available tasks. This includes viewing, adding, editing, and removing tasks, as well as assigning tasks to users and awarding coins.

- **`create_user_tasks_dashboard()`:** Generates a simplified task dashboard view specifically for regular users, allowing them to see available tasks.

- **`create_search_interface()`:** Constructs the UI for the MQTT documentation search engine, providing an input box for queries and displaying search results.

- **`create_sensor_data_ui()`:** Develops the interface for visualizing sensor data. Users can select environment, sensor, date, and hour to view graphs and average values.(Working on fake data)

- **`create_shop_ui()`:** Creates the UI for a reward shop where authenticated users can browse and purchase items using earned coins.

- **`create_chatbot_ui()`:** Builds the interface for an AI assistant chatbot, allowing users to interact and get information.

- **`create_main_interface()`:** This is the central function that orchestrates the entire application. It sets up the authentication flow (login and registration) and dynamically displays different sets of tabs (main, admin, user) based on the user's authentication status and role. It integrates all the other UI components within these tabs.

## ✨ Functionality

The UI components are connected to backend logic within the imported modules (`admin`, `searchService`, `taskLogic`, `userManager`, `sensorVisualLogic`, `chatBot`, `dbService`). These backend functions handle:
- Managing search index and status.
- Performing searches based on user input.
- Handling user authentication (login and registration).
- Managing user data, including coin balances.
- Adding, removing, and retrieving tasks.
- Visualizing sensor data.
- Processing chatbot queries and generating responses.
- Managing the reward shop and item purchases.


## 🚀 Running the Application

The notebook is designed to be executed in a Google Colab environment. The final cell `create_main_interface().launch(debug=True, share=True)` will start the Gradio web server, making the interactive user interface accessible through a public URL (when `share=True`). **Note:** The `debug=True` setting is required for the chatbot functionality to work correctly in this setup.

The UI will initially present the login and registration options, and upon successful authentication, it will reveal the tabs relevant to the user's role (admin or regular user).

# Imports and Package installations

In [1]:
import os, sys
%pip install -q -U gradio
%pip install -q pandas
%pip install -q importnb

try:
    # Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'dev' branch
    !git fetch origin -q
    !git checkout Dev -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    import gradio as gr
    import time
    from datetime import datetime
    import pandas as pd
    from importnb import Notebook
    with Notebook():
        import Indexmqtt as indx
        #import SensorDataProcessor as senDatProc
        import CloudDB as dbService
        import Admin as admin
        import SearchService as searchService
        import UserManager as userManager
        import TaskLogic as taskLogic
        import SensorVisualLogic as sensorVisualLogic
        import ChatbotLogic as chatBot

except Exception as e:
    print("❌ Setup failed:", str(e))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00
Cloning into '/content/Project-Pheonix'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 667 (delta 68), reused 22 (delta 22), pack-reused 562 (from 2)
Receiving objects: 100% (667/667), 2.07 MiB | 16.80 MiB/s, done.
Resolving deltas: 100% (372/372), done.
/content/Project-Pheonix
/content/Project-Pheonix
/content/Project-Pheonix
/content/Project-Pheonix
/content/Project-Pheonix
/content/Project-Pheonix
/content/Project-Pheonix


#Admin Panel UI

In [10]:
# Create the admin dashboard UI
def create_admin_dashboard():
    with gr.Blocks(title="Admin Dashboard") as dashboard:
        gr.Markdown("# Admin Dashboard")

        with gr.Tab("Index Management"):
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("### Top 10 Most Searched Terms")
                    top_terms_output = gr.Dataframe(
                        headers=["Rank", "Term", "Searches"],
                        row_count=10,
                        interactive=False
                    )

                    refresh_top_terms = gr.Button("Refresh Top Terms")

                with gr.Column(scale=3):
                    gr.Markdown("### Index Status")
                    index_status_md = gr.Markdown("")

                    with gr.Row():
                        reindex_button = gr.Button("Re-index Content", variant="primary")
                        refresh_index_status = gr.Button("Refresh Status")

                    index_action_output = gr.Textbox(label="Action Output", lines=2)

        # Initialize displayed data on load
        def update_top_terms():
            terms_with_counts = admin.get_top_search_terms()
            data = []
            for i, (term, count) in enumerate(terms_with_counts, 1):
                data.append([i, term, f"{count} Searches"])
            return pd.DataFrame(data, columns=["Rank", "Term", "Searches"])

        def update_index_status():
            status = admin.get_index_status()
            return f"**Word Count:** {status['word_count']}<br>**Page Count:** {status['page_count']}<br>**Last Indexed:** {status['last_indexed']}"


        # Set up event handlers
        refresh_top_terms.click(update_top_terms, outputs=top_terms_output)
        refresh_index_status.click(update_index_status, outputs=index_status_md)
        reindex_button.click(admin.reindex_content, outputs=index_action_output)


        # Initialize the UI
        dashboard.load(update_top_terms, outputs=top_terms_output)
        dashboard.load(update_index_status, outputs=index_status_md)

        return dashboard

# Admin tasks dashboard

In [3]:
# Create the admin dashboard UI
def create_admin_tasks_dashboard():
    with gr.Blocks(title="Tasks Dashboard") as tasksAdminDashboard:
        gr.Markdown("# Tasks Dashboard")

        with gr.Tab("View Tasks") as view_tab:
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📋 All Tasks")
                    view_tasks_output = gr.Dataframe(
                        headers=["Task Name", "Importance", "Value"],
                        row_count="dynamic",
                        interactive=False
                    )
                    view_refresh_button = gr.Button("🔄 Refresh Table", variant="secondary")

        with gr.Tab("Manage Tasks") as manage_tab:
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📋 Tasks Table")
                    manage_tasks_output = gr.Dataframe(
                        headers=["Task Name", "Importance", "Value"],
                        row_count="dynamic",
                        interactive=False
                    )
                    manage_refresh_button = gr.Button("🔄 Refresh Table", variant="secondary")

                with gr.Column():
                    gr.Markdown("### 🎯 Task Management")

                    # Task selection dropdown
                    task_to_manage_dropdown = gr.Dropdown(
                        label="📝 Select Task",
                        choices=[],
                        interactive=True
                    )

                    # User selection dropdown
                    user_to_complete_dropdown = gr.Dropdown(
                        label="👤 Select User to Award Coins",
                        choices=[],
                        interactive=True
                    )

                    with gr.Row():
                        complete_task_button = gr.Button("🏆 Mark as Complete", variant="primary")
                        remove_task_button = gr.Button("🗑️ Delete Task", variant="secondary")

                    management_status_output = gr.Textbox(label="Status", lines=2, interactive=False)

        with gr.Tab("Create Tasks") as create_tab:
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📋 Current Tasks")
                    create_tasks_output = gr.Dataframe(
                        headers=["Task Name", "Importance", "Value"],
                        row_count="dynamic",
                        interactive=False
                    )
                    create_refresh_button = gr.Button("🔄 Refresh Table", variant="secondary")

                with gr.Column():
                    gr.Markdown("### ➕ Add New Task")

                    task_name_input = gr.Textbox(
                        label="Task Name",
                        placeholder="Enter task name...",
                        lines=1
                    )

                    importance_dropdown = gr.Dropdown(
                        label="Importance",
                        choices=["Low", "Medium", "High"],
                        value="Medium",
                        interactive=True
                    )

                    task_value_input = gr.Textbox(
                        label="Value",
                        placeholder="Enter task value...",
                        lines=1
                    )

                    add_task_button = gr.Button("➕ Add Task", variant="primary")
                    task_status_output = gr.Textbox(label="Status", lines=1, interactive=False)

        # Initialize displayed data on load
        def update_tasks_list():
            try:
                tasks_data = taskLogic.get_all_tasks()  # This likely returns a dict with keys
                task_data = []
                task_names = []

                # Check if tasks_data is a dictionary (Firebase format)
                if isinstance(tasks_data, dict):
                    for task_key, task_value in tasks_data.items():
                        if isinstance(task_value, dict):
                            # Firebase returns {key: {task_data}} format
                            task_data.append([
                                task_value.get('name', 'Unnamed Task'),
                                task_value.get('importance', 'Medium'),
                                task_value.get('value', '')
                            ])
                            task_names.append(task_value.get('name', 'Unnamed Task'))
                        else:
                            # Handle case where task_value is a string
                            task_data.append([str(task_value), 'Medium', ''])
                            task_names.append(str(task_value))

                # Check if tasks_data is a list
                elif isinstance(tasks_data, list):
                    for task in tasks_data:
                        if isinstance(task, dict):
                            task_data.append([
                                task.get('name', 'Unnamed Task'),
                                task.get('importance', 'Medium'),
                                task.get('value', '')
                            ])
                            task_names.append(task.get('name', 'Unnamed Task'))
                        else:
                            task_data.append([str(task), 'Medium', ''])
                            task_names.append(str(task))

                # Handle None or empty data
                else:
                    print(f"Unexpected data format from get_all_tasks(): {type(tasks_data)}")
                    print(f"Data content: {tasks_data}")

                df = pd.DataFrame(task_data, columns=["Task Name", "Importance", "Value"])
                users_list = userManager.get_all_users()
                return df, gr.update(choices=task_names), gr.update(choices=users_list)

            except Exception as e:
                print(f"Error in update_tasks_list: {e}")
                print(f"Raw data from get_all_tasks(): {taskLogic.get_all_tasks()}")  # Debug line
                return pd.DataFrame([], columns=["Task Name", "Importance", "Value"]), gr.update(choices=[]), gr.update(choices=[])
        def refresh_table():
            """Refresh all tables across all tabs, user list, and update status message"""
            df, task_choices, user_choices = update_tasks_list()
            return df, df, df, task_choices, user_choices, "🔄 All tables and user list refreshed successfully!"

        def complete_task_and_refresh(task_name, username):
             """Complete task and return all required outputs"""
             status_message = complete_task(task_name, username)
             df, task_choices, user_choices = update_tasks_list()
             # Return 6 values: status, 3 dataframes, 2 dropdowns
             return status_message, df, df, df, task_choices, user_choices

        def complete_task(task_name, username):
            """Mark a task as complete and award coins to the user"""
            if not task_name:
                return "❌ Please select a task to complete!"
            if not username:
                return "❌ Please select a user to award coins to!"


            # FIX: Get the specific task by name, not all tasks
            task = taskLogic.get_task(task_name)  # You need this method
            # OR if you need to search through all tasks:
            tasks_data = taskLogic.get_all_tasks()
            task = None
            task_value = 0

            if isinstance(tasks_data, dict):
                for task_key, task_data in tasks_data.items():
                    if isinstance(task_data, dict) and task_data.get('name') == task_name:
                        task = task_data
                        task_value = int(task_data.get('value', 0)) if task_data.get('value', '').isdigit() else 0
                        break

            if task is None:
                return "❌ Task not found!"

            # Update user coins
            success, message = userManager.update_user_coins(username, task_value)

            if success:
                # Remove the completed task
                taskLogic.remove_task(task_name)
                return f"🏆 Task: {task_name} completed by {username}!"
            else:
                return f"❌ Failed to complete task: {message}"

        def add_task(task_name, importance, task_value):
            if not task_name.strip():
                return (
                    gr.update(), gr.update(), gr.update(), gr.update(), gr.update(),
                    "❌ Task name cannot be empty!",
                    gr.update(value=""), gr.update(), gr.update(value="")
                )
            if not task_value.strip() or not task_value.strip().isdigit():
                return (
                    gr.update(), gr.update(), gr.update(), gr.update(), gr.update(),
                    "❌ Task value cannot be empty and must contain only numbers!",
                    gr.update(value=""), gr.update(), gr.update(value="")
                 )
            task_data = {
                'name': task_name.strip(),
                'importance': importance,
                'value': task_value.strip()
            }

            taskLogic.add_task(task_data)
            df, task_choices, user_choices = update_tasks_list()

            return (
                df, df, df, task_choices, user_choices,  # Update all tables and dropdowns
                f"✅ Task '{task_name}' added successfully!",
                gr.update(value=""), gr.update(value="Medium"), gr.update(value="")
            )

        def remove_task(task_name):
            if not task_name:
                df, task_choices, user_choices = update_tasks_list()
                return df, df, task_choices, user_choices, "❌ Please select a task to remove!"

            if taskLogic is None:
                df, task_choices, user_choices = update_tasks_list()
                return df, df, task_choices, user_choices, "❌ taskLogic module not available!"

            try:
                taskLogic.remove_task(task_name)
                df, task_choices, user_choices = update_tasks_list()
                return df, df, task_choices, user_choices, f"✅ Task '{task_name}' removed successfully!"
            except Exception as e:
                df, task_choices, user_choices = update_tasks_list()
                return df, df, task_choices, user_choices, f"❌ Error removing task: {str(e)}"

        # Set up event handlers
        view_refresh_button.click(
            refresh_table,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown, management_status_output]
        )

        manage_refresh_button.click(
            refresh_table,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown, management_status_output]
        )

        create_refresh_button.click(
            refresh_table,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown, task_status_output]
        )

        complete_task_button.click(
            complete_task_and_refresh,
            inputs=[task_to_manage_dropdown, user_to_complete_dropdown],
            outputs=[management_status_output, view_tasks_output, manage_tasks_output,
                    create_tasks_output, task_to_manage_dropdown, user_to_complete_dropdown]
        )
        remove_task_button.click(
            remove_task,
            inputs=[task_to_manage_dropdown],
            outputs=[view_tasks_output, manage_tasks_output, task_to_manage_dropdown,
                    user_to_complete_dropdown, management_status_output]
        )

        add_task_button.click(
            add_task,
            inputs=[task_name_input, importance_dropdown, task_value_input],
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown,
                    task_status_output, task_name_input, importance_dropdown, task_value_input]
        )

        # Add tab change event handlers to refresh data when switching tabs
        def refresh_on_tab_change():
            """Refresh all data when switching tabs"""
            df, task_choices, user_choices = update_tasks_list()
            return df, df, df, task_choices, user_choices

        # Initialize the UI
        tasksAdminDashboard.load(
            refresh_on_tab_change,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown]
        )

        view_tab.select(
            fn=refresh_on_tab_change,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown]
        )

        manage_tab.select(
            fn=refresh_on_tab_change,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown]
        )

        create_tab.select(
            fn=refresh_on_tab_change,
            outputs=[view_tasks_output, manage_tasks_output, create_tasks_output,
                    task_to_manage_dropdown, user_to_complete_dropdown]
        )

        return tasksAdminDashboard

#tasksUserDashboard = create_admin_tasks_dashboard()
#tasksUserDashboard.launch(debug=True, share=True)

#User tasks Dashbord

In [4]:
# Create the admin dashboard UI
def create_user_tasks_dashboard():
    with gr.Blocks(title="Tasks Dashboard") as tasksUserDashboard:
        gr.Markdown("# Tasks Dashboard")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📋 All Tasks")
                view_tasks_output = gr.Dataframe(
                    headers=["Task Name", "Importance", "Value"],
                    row_count="dynamic",
                    interactive=False
                )
                view_refresh_button = gr.Button("🔄 Refresh Table", variant="secondary")

        # Initialize displayed data on load
        def update_tasks_list():
            try:
                tasks_data = taskLogic.get_all_tasks()
                task_data = []
                task_names = []

                # Check if tasks_data is a dictionary (Firebase format)
                if isinstance(tasks_data, dict):
                    for task_key, task_value in tasks_data.items():
                        if isinstance(task_value, dict):
                            # Firebase returns {key: {task_data}} format
                            task_data.append([
                                task_value.get('name', 'Unnamed Task'),
                                task_value.get('importance', 'Medium'),
                                task_value.get('value', '')
                            ])
                            task_names.append(task_value.get('name', 'Unnamed Task'))
                        else:
                            # Handle case where task_value is a string
                            task_data.append([str(task_value), 'Medium', ''])
                            task_names.append(str(task_value))

                # Check if tasks_data is a list
                elif isinstance(tasks_data, list):
                    for task in tasks_data:
                        if isinstance(task, dict):
                            task_data.append([
                                task.get('name', 'Unnamed Task'),
                                task.get('importance', 'Medium'),
                                task.get('value', '')
                            ])
                            task_names.append(task.get('name', 'Unnamed Task'))
                        else:
                            task_data.append([str(task), 'Medium', ''])
                            task_names.append(str(task))

                # Handle None or empty data
                else:
                    print(f"Unexpected data format from get_all_tasks(): {type(tasks_data)}")
                    print(f"Data content: {tasks_data}")
                    task_data = []
                    task_names = []

                df = pd.DataFrame(task_data, columns=["Task Name", "Importance", "Value"])

                try:
                    users_list = userManager.get_all_users()
                except Exception as user_error:
                    print(f"Error getting users: {user_error}")
                    users_list = []

                return df, gr.update(choices=task_names), gr.update(choices=users_list)

            except Exception as e:
                print(f"Error in update_tasks_list: {e}")
                empty_df = pd.DataFrame([], columns=["Task Name", "Importance", "Value"])
                return empty_df, gr.update(choices=[]), gr.update(choices=[])

        def refresh_table():
            """Refresh the table"""
            df, task_choices, user_choices = update_tasks_list()
            return df

        # Set up event handlers
        view_refresh_button.click(
            refresh_table,
            outputs=[view_tasks_output]
        )

        # Initialize the UI
        def refresh_on_load():
            """Refresh data on load"""
            df, task_choices, user_choices = update_tasks_list()
            return df

        tasksUserDashboard.load(
            refresh_on_load,
            outputs=[view_tasks_output]
        )

        return tasksUserDashboard

#tasksUserDashboard = create_user_tasks_dashboard()
#tasksUserDashboard.launch(debug=True, share=True)

#Search Engine UI

In [5]:
# Create the Gradio interface for the search engine
def create_search_interface():
    with gr.Blocks() as search_interface:
        gr.Markdown("""
        # 🔍 MQTT Documentation Search
        """)

        with gr.Row():
            with gr.Column(scale=6):
                search_input = gr.Textbox(
                    placeholder="Search for MQTT topics, concepts, or features...",
                    label="",
                    show_label=False,
                    lines=1
                )
            with gr.Column(scale=1):
                search_button = gr.Button("🔍 Search", variant="primary", size="lg")

        # Information about search capabilities
        with gr.Accordion("Search Tips", open=False):
            gr.Markdown("""
            This search engine uses word stemming to find related word forms.
            - Searching for "connect" will also find "connecting" and "connection"
            - Try multiple keywords to narrow results (e.g., "mqtt broker")
            - Results are ranked by relevance to your query
            """)

        # Area for search results - using HTML instead of Textbox
        search_results = gr.HTML(
            "<p>🔎 Enter your search terms above</p><p>Search for MQTT related terms like 'broker', 'publish', 'subscribe', etc.</p>"
        )

        # Connect the search button to the search function
        search_button.click(
            fn=searchService.search_word,
            inputs=search_input,
            outputs=search_results
        )

        # Add handling for pressing Enter key in the search box
        search_input.submit(
            fn=searchService.search_word,
            inputs=search_input,
            outputs=search_results
        )

    return search_interface



# Sensor UI

In [6]:
def create_sensor_data_ui():
  with gr.Blocks() as sensor:
      gr.Markdown("## 📊 Sensor Data Visualization")
      with gr.Row():
        with gr.Column(scale=1): pass
        with gr.Column(scale=2):
            rs = gr.Dropdown(choices=sensorVisualLogic.enviorment, value='indoor', label="Environment")
            rs_sensors = gr.Dropdown(choices=sensorVisualLogic.sensors['indoor'], interactive=True, label="Sensor")
            rs_dates = gr.Dropdown(choices=sensorVisualLogic.dates, interactive=True, label='Date')
            rs_hours = gr.Dropdown(choices=sensorVisualLogic.initial_hours, interactive=True, label='Hour')
            rs.change(fn=sensorVisualLogic.rs_change, inputs=rs, outputs=rs_sensors)
            rs_dates.change(fn=sensorVisualLogic.date_change, inputs=rs_dates, outputs=rs_hours)
        with gr.Column(scale=1): pass

      with gr.Row():
        with gr.Column(scale=1): pass
        with gr.Column(scale=2):
            submit_btn = gr.Button("Generate", variant="primary")
        with gr.Column(scale=1): pass

      with gr.Column():
        plot_output = gr.Plot(label="Sensor Data Graph")
        avg_textbox = gr.Textbox(label="Average Value")

      submit_btn.click(
        fn=sensorVisualLogic.plot_graph,
        inputs=[rs, rs_sensors, rs_dates, rs_hours],
        outputs=[plot_output, avg_textbox]
      )
  return sensor


#Shop UI

In [7]:
# Global variable to track authenticated user
current_authenticated_user = None

# Function to create the Shop UI
def create_shop_ui():
    global current_authenticated_user

    # Get the current user's data
    username = current_authenticated_user
    user_data = dbService.get_user_by_username(username)
    initial_coins = dbService.get_coins_from_db(username) if user_data else 0

    # Create the Gradio interface
    with gr.Blocks(theme=gr.themes.Citrus()) as shop:
        gr.Markdown("## 🛒 Shop")

        # User greeting and current coin display
        with gr.Row():
            user_greeting = gr.Markdown(value="Welcome **...**")
            current_coins = gr.Markdown(value="Coins: **0** 💰")

        # Checkbox group for selecting rewards
        cart = gr.State([])
        items_to_add = gr.CheckboxGroup(
            ["Free Coffee ☕️ :50 coins", "Free Meal 🍔 :100 coins", "Pizza Party 🍕 :200 coins", "Water Park 💧 :300 coins", "Day Off 😄 :400 coins"],
            label="Choose Items to Add"
        )

        def get_user_greeting():
            return f"Welcome **{current_authenticated_user}**"

        update_greeting_button = gr.Button("🔄 Update Greeting", visible=False)
        update_greeting_button.click(
          fn=get_user_greeting,
          outputs=[user_greeting]
)

        with gr.Row():
            add_button = gr.Button("➕ Add Items to Cart", variant="primary", size="lg")
            delete_button = gr.Button("❌ Clear Cart", variant="secondary")
        cart_display = gr.Markdown("🛒 **Cart is empty**")
        cart_size = gr.Number(label="Cart Size", interactive=False)
        checkout_result = gr.Markdown("")


        # Function to update user's coins using userManager
        def update_user_coins(greeting_display, new_coins):
            try:
              username = greeting_display.split("**")[1]
            except:
              return False  # Failed to extract username
            user_data = dbService.get_user_by_username(username)
            if user_data:
                # Update the user in the database
                dbService.change_coins_for_user(username,-new_coins)
                return True
            return False

        # Returns a string of all items of the cart, or if it's empty returns empty cart
        def format_cart(cart_list):
            if not cart_list:
                return "🛒 **Cart is empty**"
            return "🛒 **Your Cart:**\n" + "\n".join([f"- {item}" for item in cart_list])

        # Adds items to the cart
        def add_items(new_items, previous_cart):
            new_cart = previous_cart + new_items
            return new_cart, format_cart(new_cart), len(new_cart)

        # Initiates checkout, sums the total cost of all items, returns what was bought and subtracts from the user's coins
        def checkout(cart_items, current_coin_display, greeting_display):
            try:
              username = greeting_display.split("**")[1]
            except:
              return "❌ Error extracting username. Please try again.", cart_items, format_cart(cart_items), len(cart_items), current_coin_display

            if not cart_items:
                return "❌ Your cart is empty!", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{dbService.get_coins_from_db(username)}** 💰"
            try:
                current_coin_value = int(current_coin_display.split("**")[1].split()[0])
            except:
                current_coin_value = 0
            messages = ["🧾 **Checkout Summary:**"]
            total_cost = 0

            for item in cart_items:
                match item:
                    case "Free Coffee ☕️ :50 coins":
                        messages.append("☕️ Coffee - 50 coins")
                        total_cost += 50
                    case "Free Meal 🍔 :100 coins":
                        messages.append("🍔 Meal - 100 coins")
                        total_cost += 100
                    case "Pizza Party 🍕 :200 coins":
                        messages.append("🍕 Pizza - 200 coins")
                        total_cost += 200
                    case "Water Park 💧 :300 coins":
                        messages.append("💧 Water Park - 300 coins")
                        total_cost += 300
                    case "Day Off 😄 :400 coins":
                        messages.append("😄 Day Off - 400 coins")
                        total_cost += 400
                    case _:
                        messages.append(f"❓ Unknown item: {item}")

            # Checks if the total sum of items in the cart is smaller than the amount of coins the user has
            if total_cost > current_coin_value:
                return "❌ Not enough coins to complete the purchase!", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{current_coin_value}** 💰"

            # Update user's coins
            new_coins = current_coin_value - total_cost
            if update_user_coins(greeting_display, total_cost):
                messages.append(f"\n💰 **Total Cost:** {total_cost} coins")
                messages.append(f"💰 **Remaining Coins:** {new_coins} coins")
                return "\n".join(messages), [], format_cart([]), 0, f"Coins: **{new_coins}** 💰"
            else:
                return "❌ Error updating coins. Please try again.", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{current_coin_value}** 💰"

        # Clears the cart
        def delete_cart(cart_items):
            return [], "🛒 **Cart is empty**", 0

        with gr.Row():
            checkout_button = gr.Button("✅ Checkout", variant="secondary")

        # Event handlers
        add_button.click(
            fn=add_items,
            inputs=[items_to_add, cart],
            outputs=[cart, cart_display, cart_size]
        )

        checkout_button.click(
            fn=checkout,
            inputs=[cart, current_coins, user_greeting],
            outputs=[checkout_result, cart, cart_display, cart_size, current_coins]
        )

        delete_button.click(
            fn=delete_cart,
            inputs=[cart],
            outputs=[cart, cart_display, cart_size]
        )

    return shop, user_greeting, current_coins

#Chatbot UI

In [8]:
def create_chatbot_ui():
    with gr.Blocks() as chatbot_interface:
        gr.Markdown("## 🤖 AI Assistant Chatbot")

        # Input section
        with gr.Row():
            user_input = gr.Textbox(
                placeholder="Type your message here...",
                label="",
                show_label=False,
                lines=1,
                max_lines=3
            )
        with gr.Row():
            send_button = gr.Button("📤 Send", variant="primary", size="lg")

        # Chat tips accordion
        with gr.Accordion("💡 Chat Tips", open=False):
            gr.Markdown("""
            **How to use the chatbot:**
            - Type your questions or messages in the text box below
            - Press Enter or click Send to submit your message
            - The bot will respond to your queries using Google's Gemini AI
            - Use Clear Chat to start a fresh conversation

            **Example questions:**
            - "How does MQTT work?"
            - "Explain sensor data visualization"
            - "What features are available in this system?"
            - "Help me understand cloud computing"
            """)

        # Chat history display
        with gr.Row():
            chatbot_display = gr.Chatbot(
                type='messages',
                label="Chat History",
                height=400,
                show_label=True,
                avatar_images=["👤", "🤖"]
            )

        # Clear button
        with gr.Row():
            clear_button = gr.Button("🗑️ Clear Chat", variant="secondary")

        # FIXED: Use the correct function from ChatbotLogic
        def clear_chat():
            return []

        # Event handlers - FIXED to use the new chatbot function
        send_button.click(
            fn=chatBot.handle_chat_message,  # Use the new function
            inputs=[user_input, chatbot_display],
            outputs=[user_input, chatbot_display]
        )

        user_input.submit(
            fn=chatBot.handle_chat_message,  # Use the new function
            inputs=[user_input, chatbot_display],
            outputs=[user_input, chatbot_display]
        )

        clear_button.click(
            fn=clear_chat,
            outputs=[chatbot_display]
        )

    return chatbot_interface

#Unified UI

In [11]:
# Create the main interface with authentication flow
def create_main_interface():
    global current_authenticated_user

    # Create the interface
    with gr.Blocks(title="Cloud Project - Phoenix", theme=gr.themes.Base()) as main_interface:
        gr.Markdown("# 🐦 Phoenix Team Project")

        # Authentication state variables
        authenticated_user = gr.State(None)
        is_admin = gr.State(False)
        is_logged_in = gr.State(False)

        # Main tabs for unauthenticated users (initially visible)
        with gr.Column(visible=True) as main_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome to Our Phoenix Team Project!

                    ## Available Features:
                    - Search Engine for MQTT
                    - Sensor Data Visualization
                    - Reward Shop (for users)
                    - Task Dashboard
                    - Task Management (for admin users)
                    - Admin Dashboard (for admin users)

                    Please log in to access all features.
                    """)

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("🔐 Login"):
                    gr.Markdown("## 🔐 Login")
                    gr.Markdown("- username : admin , password : admin | for an account with admin privileges")
                    gr.Markdown("- username : user , password : user | for an account with user privileges")
                    with gr.Row():
                        login_username = gr.Textbox(label="Username")
                        login_password = gr.Textbox(label="Password", type="password")
                    login_button = gr.Button("Login", variant="primary")
                    login_output = gr.Textbox(label="Login Status", lines=1)

        # Admin tabs (initially hidden)
        with gr.Column(visible=False) as admin_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome Admin! 🔥

                    You have successfully logged in as an **Administrator**.

                    ## Admin Features Available:
                    - 👨‍💼 **Admin Dashboard** - Manage search terms, index status, and daily tasks
                    - 📝 **User Registration** - Create new user accounts and admin accounts
                    - 🔍 **Search Engine** - Full access to MQTT documentation search
                    - 📊 **Sensor Data** - View and analyze sensor data visualizations
                    - 📋 **Tasks** - Manage the tasks in the task dashboard
                    - 🤖 **AI Assistant Chatbot**


                    Select any tab above to get started!
                    """)

                with gr.TabItem("👨‍💼 Admin Dashboard"):
                    create_admin_dashboard()

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("📊 Sensor Data"):
                    create_sensor_data_ui()

                with gr.TabItem("📋 Tasks"):
                    create_admin_tasks_dashboard()

                with gr.TabItem("🤖 Chatbot"):
                    create_chatbot_ui()

                with gr.TabItem("📝 Register New Users"):
                    gr.Markdown("## 📝 Register New User (Admin Only)")
                    with gr.Row():
                        reg_username = gr.Textbox(label="Username")
                        reg_password = gr.Textbox(label="Password", type="password")
                        reg_confirm_password = gr.Textbox(label="Confirm Password", type="password")
                    reg_is_admin = gr.Checkbox(label="Register as Admin")
                    register_button = gr.Button("Register User", variant="primary")
                    register_output = gr.Textbox(label="Registration Status", lines=1)

        # Regular user tabs (initially hidden)
        with gr.Column(visible=False) as user_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome User! 🎉

                    You have successfully logged in to the Phoenix Team Project.

                    ## Available Features:
                    - 🔍 **Search Engine** - Search through MQTT documentation with advanced word stemming
                    - 📊 **Sensor Data** - Visualize indoor and outdoor sensor data with interactive charts
                    - 🛒 **Shop** - Use your coins to purchase rewards and benefits
                    - 📋 **Tasks** - View the tasks in the task dashboard
                    - 🤖 **AI Assistant Chatbot**

                    ## How to Get Started:
                    1. **Search Engine**: Try searching for terms like "broker", "publish", or "subscribe"
                    2. **Sensor Data**: Select environment, sensor type, date, and hour to view data
                    3. **Shop**: Browse available rewards and add items to your cart
                    4. **Chatbot**: Chat with the AI assistant to get answers to your questions

                    ## Tips:
                    - The search engine uses word stemming (searching "connect" finds "connection", "connecting")
                    - Sensor data is updated in real-time from our IoT devices
                    - You start with 2500 coins in the shop system

                    Select any tab above to explore the features!
                    """)

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("📊 Sensor Data"):
                    create_sensor_data_ui()

                with gr.TabItem("📋 Tasks"):
                    create_user_tasks_dashboard()

                with gr.TabItem("🤖 Chatbot"):
                    create_chatbot_ui()

                with gr.TabItem("🛒 Shop"):
                    shop_ui, user_greeting, current_coins_display  = create_shop_ui()
        # Logout section (initially hidden)
        with gr.Row(visible=False) as logout_section:
            logout_button = gr.Button("🔓 Logout", variant="secondary", size="lg")

        # Event handler functions
        def handle_login(username, password):
            global current_authenticated_user
            message, user, admin_status = userManager.login(username, password)
            if user is not None:
                current_authenticated_user = user  # Set global user variable
                greeting = f"Welcome **{user}**"
                coins = dbService.get_coins_from_db(user)
                coin_display = gr.update(value=f"Coins: **{coins}** 💰")

                if admin_status:
                    return (
                        message, user, admin_status, True,
                        gr.update(visible=False), gr.update(visible=True),
                        gr.update(visible=False), gr.update(visible=True),
                        gr.update(value="Welcome ..."),
                        gr.update(value="Coins: 0 💰")
                    )
                else:
                    return (
                        message, user, admin_status, True,
                        gr.update(visible=False), gr.update(visible=False),
                        gr.update(visible=True), gr.update(visible=True),
                        greeting,
                        coin_display
                    )
            else:
                current_authenticated_user = None  # Clear global user variable
                return (
                    message, None, False, False,
                    gr.update(visible=True), gr.update(visible=False),
                    gr.update(visible=False), gr.update(visible=False),
                    gr.update(value="Welcome ..."),
                    gr.update(value="Coins: 0 💰")
                )

        def handle_register(username, password, confirm, is_admin_val):
            message, user, admin_status = userManager.register_account(username, password, confirm, is_admin_val)
            return message

        def handle_logout():
            global current_authenticated_user
            current_authenticated_user = None  # Clear global user variable
            return (
                None, False, False,
                gr.update(visible=True), gr.update(visible=False),
                gr.update(visible=False), gr.update(visible=False),
                gr.update(value=""), gr.update(value=""), gr.update(value="")
            )

        # Event handlers
        login_button.click(
            fn=handle_login,
            inputs=[login_username, login_password],
            outputs=[login_output, authenticated_user, is_admin, is_logged_in,
                    main_tabs, admin_tabs, user_tabs, logout_section, user_greeting, current_coins_display]
        )

        register_button.click(
            fn=handle_register,
            inputs=[reg_username, reg_password, reg_confirm_password, reg_is_admin],
            outputs=[register_output]
        )

        logout_button.click(
            fn=handle_logout,
            outputs=[authenticated_user, is_admin, is_logged_in,
                    main_tabs, admin_tabs, user_tabs, logout_section,
                    login_username, login_password, login_output]
        )

    return main_interface

# Create the main interface
main_interface = create_main_interface()
main_interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://babb81273e664c65a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://babb81273e664c65a8.gradio.live


### 🔗 Project Links (Dev Branch)

**GitHub Repository (Dev Branch):**
https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/tree/Dev

**Links to Project Notebooks:**
- **Main Notebook:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/UI/HW2.ipynb
- **Admin Logic:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/Admin.ipynb
- **Chatbot Logic:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/ChatbotLogic.ipynb
- **Cloud Database:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/CloudDB.ipynb
- **MQTT Indexing:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/Indexmqtt.ipynb
- **Search Service:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/SearchService.ipynb
- **Sensor Data Processor:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/SensorDataProcessor.ipynb
- **Sensor Visualization Logic:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/SensorVisualLogic.ipynb
- **Task Logic:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/TaskLogic.ipynb
- **User Manager:** https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/UserManager.ipynb